# MiniFlow

In [1]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # a node has a list of inbound nodes
        self.inbound_nodes = inbound_nodes
        # Node(s) which this node passes the values to
        self.outbound_nodes = []
        
        # for each inbound node in this node, this node will become an outbound node
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
            pass
        
        # Each node will calculate a value in each pass
        self.value = None
        pass
    
    def forward(self):
        """
        Forward propagation
        Calculate the value of the node based on the
        `inbound_nodes` and store the value in `self.value`
        """
        raise NotImplemented

In [3]:
class Input(Node):
    def __init__(self):
        Node.__init__(self)
        pass
    
    def forward(self, value=None):
        if value is not None:
            self.value = value            

In [9]:
class Add(Node):
    def __init__(self, *inputs):
        Node.__init__(self, inputs)
        pass
    
    def forward(self):
        self.value = 0
        for node in self.inbound_nodes:
            self.value += node.value

In [8]:
class Mul(Node):
    def __init__(self, *inputs):
        Node.__init__(self, inputs)
        pass
    
    def forward(self):
        self.value = 1
        for node in self.inbound_nodes:
            self.value *= node.value        

In [10]:
import numpy as np
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])
        pass
    
    def forward(self):
        if len(self.inbound_nodes) < 3:
            self.value = None
        else:
            self.value = np.dot(self.inbound_nodes[0].value, self.inbound_nodes[1].value) + self.inbound_nodes[2].value

In [6]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

In [7]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value